<a href="https://colab.research.google.com/github/kumarashishraj/CNN-Optimisation-Keras-Tunner/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install keras-tuner

In [15]:
import tensorflow as tf
from tensorflow import keras
import numpy as np 

In [16]:
print (tf.__version__)

2.6.0


In [17]:
fashion_minst = keras.datasets.fashion_mnist

In [18]:
(train_images, train_labels), (test_images, test_labels ) = fashion_minst.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [19]:
train_images = train_images/255.0
test_images = test_images/255.0

In [20]:
train_images[0].shape

(28, 28)

In [21]:
train_images = train_images.reshape (len (train_images), 28, 28, 1)
test_images = test_images.reshape (len (test_images), 28, 28, 1)

In [22]:
def build_model (hp):
  model = keras.Sequential(
      [keras.layers.Conv2D(
          filters = hp.Int ('conv_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice ('conv_1_kernel', values = [3, 5]),
          activation = 'relu',
          input_shape = (28, 28, 1)
      ),
       keras.layers.Conv2D(
          filters = hp.Int ('conv_1_filter', min_value = 32, max_value = 128, step = 16),
          kernel_size = hp.Choice ('conv_2_kernel', values = [3, 5]),
          activation = 'relu'
      ),
       keras.layers.Flatten(),
       keras.layers.Dense(
           units = hp.Int ('dense_1_units', min_value = 32, max_value = 128, step = 16),
           activation = 'relu'
       ),
       keras.layers.Dense (10, activation = 'softmax')      
  ])

  model.compile (optimizer = keras.optimizers.Adam(hp.Choice ('learning_rate', values = [1e-2, 1e-3])),
                 loss = 'sparse_categorical_crossentropy',
                 metrics = ['accuracy'])
  return model;


In [23]:
!pip install keras-tuner --upgrade

In [24]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [25]:
tuner_search = RandomSearch (build_model, 
                             objective = 'val_accuracy',
                             max_trials = 5,
                             directory = 'output', 
                             project_name = "Minst Fashion")

In [26]:
tuner_search.search (train_images, train_labels, epochs = 3, validation_split = 0.1)

Trial 5 Complete [00h 01m 00s]
val_accuracy: 0.8553333282470703

Best val_accuracy So Far: 0.9111666679382324
Total elapsed time: 00h 07m 53s
INFO:tensorflow:Oracle triggered exit


In [27]:
model = tuner_search.get_best_models(num_models = 1)[0]

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 30976)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2478160   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,517,562
Trainable params: 2,517,562
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.fit (train_images, train_labels, epochs = 10, validation_split=0.1, initial_epoch= 3)

Epoch 4/10
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1600 - accuracy: 0.9405 - val_loss: 0.2610 - val_accuracy: 0.9117
Epoch 5/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1253 - accuracy: 0.9543 - val_loss: 0.2710 - val_accuracy: 0.9118
Epoch 6/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0943 - accuracy: 0.9654 - val_loss: 0.3031 - val_accuracy: 0.9130
Epoch 7/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0738 - accuracy: 0.9728 - val_loss: 0.3563 - val_accuracy: 0.9113
Epoch 8/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0557 - accuracy: 0.9791 - val_loss: 0.3955 - val_accuracy: 0.9130
Epoch 9/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0478 - accuracy: 0.9825 - val_loss: 0.4405 - val_accuracy: 0.9152
Epoch 10/10
1688/1688 [==============================] - 13s 8ms/step - loss: 0.0386 - accuracy: 0.9854 - val_loss: 0.4630 - val_a